In [23]:
from biocrnpyler import *
import pylab as plt
import numpy as np
from bokeh.layouts import row

import warnings
import bokeh.io
import bokeh.plotting



class TranscriptionSwitch(Mechanism):
    
    def __init__(self, name="transcription_switch", mechanism_type="transcription"):
        
            Mechanism.__init__(self, name=name, mechanism_type=mechanism_type)
    
    def update_species(self, switch_off, activator, inhibitor, rnap, transcript, rnaseH, **keywords):

        return [switch_off, transcript, activator, inhibitor, rnap, rnaseH] 
    
    
    def update_reactions(self, switch_off, activator, inhibitor, component, part_id, rnap, rnaseH, transcript, **keywords):
        

            
        switch_on = ComplexSpecies([switch_off, activator], name = str(switch_off).replace("_OFF","_ON"))
        A_I_complex = ComplexSpecies([inhibitor, activator],name = str(switch_off).replace("_OFF","")+"_AI")
        
        ktx = component.get_parameter("ktx", part_id = part_id, mechanism = self)
        kleak = component.get_parameter("kleak", part_id = part_id, mechanism = self)
        kdeg = component.get_parameter("kdeg", part_id = part_id, mechanism = self)
        
        kb_tx = component.get_parameter("kb_tx", part_id = part_id, mechanism = self)
        kb_leak = component.get_parameter("kb_leak", part_id = part_id, mechanism = self)
        kb_deg = component.get_parameter("kb_deg", part_id = part_id, mechanism = self)
        
        kM_tx = component.get_parameter("kM_tx", part_id = part_id, mechanism = self)
        kM_leak = component.get_parameter("kM_leak", part_id = part_id, mechanism = self)
        kM_deg = component.get_parameter("kM_deg", part_id = part_id, mechanism = self)
        
        kon = component.get_parameter("kon", part_id = part_id, mechanism = self)
        koff = component.get_parameter("koff", part_id = part_id, mechanism = self)
        ka = component.get_parameter("ka", part_id = part_id, mechanism = self)
            
        #ku_tx = (kb_tx + ktx) / kM_tx
        #ku_leak = (kb_leak + kleak) / kM_leak
        #ku_deg = (kb_deg + kdeg) / kM_deg
        
        ku_tx = 0.1
        ku_leak = 0.1
        ku_deg = 0.1
        
        reaction_activation = Reaction(inputs = [switch_off, activator], outputs = [switch_on], 
                            k = kon )
        
        reaction_deactivation = Reaction(inputs = [switch_on, inhibitor], outputs = [switch_off, A_I_complex], 
                            k = koff )
        
        reaction_complex = Reaction(inputs = [activator, inhibitor], outputs = [A_I_complex], k = ka)
        
        reaction_tx_1 = Reaction(inputs = [switch_on, rnap], outputs = [ComplexSpecies([rnap, switch_on])], k = kb_tx, k_rev = ku_tx )
        
        reaction_leak_1 = Reaction(inputs = [switch_off, rnap], outputs = [ComplexSpecies([rnap, switch_off])], k = kb_leak, k_rev = ku_leak)
        
        reaction_deg_1 = Reaction(inputs = [A_I_complex, rnaseH], outputs = [ComplexSpecies([rnaseH, A_I_complex])], k = kb_deg, k_rev = ku_deg)
        
        reaction_tx_2 = Reaction(inputs = [ComplexSpecies([rnap, switch_on])], outputs = [switch_on, transcript, rnap], k = ktx)
        
        reaction_leak_2 = Reaction(inputs = [ComplexSpecies([rnap, switch_off])], outputs = [switch_off, transcript, rnap], k = kleak)
        
        reaction_deg_2 = Reaction(inputs = [ComplexSpecies([rnaseH, A_I_complex])], outputs = [rnaseH, activator], k = kdeg)
        
        return [reaction_activation, reaction_deactivation, reaction_complex, reaction_tx_1, reaction_tx_2, reaction_leak_1, reaction_leak_2, reaction_deg_1, reaction_deg_2]
    
    

In [24]:
class Switch(Promoter):
    def __init__(self, name, transcript, activator, inhibitor, rnap="RNAP",rnaseH="RNAseH" , **keywords):
        #Set the Regulator
        #Component.set_species(species, material_type = None, attributes = None)
        # is a helper function that allows the input to be a Species, string, or Component.
        
        self.activator = self.set_species(activator, material_type = "dna") 
        self.inhibitor = self.set_species(inhibitor, material_type = "rna")
        self.transcript = self.set_species(transcript, material_type = "rna")
        self.switch_off = self.set_species(str(name)+"_OFF")
        self.rnap = self.set_species(rnap, material_type = "protein")
        self.rnaseH = self.set_species(rnaseH, material_type = "protein")
        
        #Mechanisms are inherited from the Mixture unless set specifically in self.default_mechanisms.
        custom_mechanisms = {"transcription": TranscriptionSwitch()}
        
        #Always call the superclass __init__() with **keywords passed through
        Promoter.__init__(self, name = name, transcript = transcript, mechanisms = custom_mechanisms, **keywords)

    def update_species(self, **keywords):
        #Mechanisms are stored in an automatically created dictionary: mechanism_type --> Mechanism Instance.
        mech_tx = self.mechanisms["transcription"]
        
        species = [] #A list of species must be returned
        species += mech_tx.update_species(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                          rnap = self.rnap, rnaseH = self.rnaseH)
        
        return species

    def update_reactions(self, **keywords):
        mech_tx = self.mechanisms["transcription"]
        
        reactions = [] #a list of reactions must be returned
        reactions += mech_tx.update_reactions(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                              rnap = self.rnap, rnaseH = self.rnaseH, component = self, part_id = "Switch", **keywords)
        return reactions

    

class Source(Promoter):
    def __init__(self, name, transcript, rnap="RNAP", **keywords):
        #Set the Regulator
        #Component.set_species(species, material_type = None, attributes = None)
        # is a helper function that allows the input to be a Species, string, or Component
        self.dna = self.set_species(name)
        self.rnap = self.set_species(rnap, material_type = "protein")
        #Mechanisms are inherited from the Mixture unless set specifically in self.default_mechanisms.
        custom_mechanisms = {"transcription": Transcription_MM()}
        
        #Always call the superclass __init__() with **keywords passed through
        Promoter.__init__(self, name = name, transcript = transcript, mechanisms = custom_mechanisms, **keywords)

    def update_species(self, **keywords):
        #Mechanisms are stored in an automatically created dictionary: mechanism_type --> Mechanism Instance.
        mech_tx = self.mechanisms["transcription"]
        
        species = [] #A list of species must be returned
        species += mech_tx.update_species(dna = self.dna, transcript = self.transcript, rnap = self.rnap)
        
        return species

    def update_reactions(self, **keywords):
        mech_tx = self.mechanisms["transcription"]
        
        reactions = [] #a list of reactions must be returned
        reactions += mech_tx.update_reactions(dna = self.dna, transcript = self.transcript, 
                                              rnap = self.rnap, component = self, part_id = "Source", **keywords)
        return reactions    
    
    
    

In [ ]:
"""
def GeneletGate(name, out, on_1 = None, on_2 = None, off_1 = None, off_2 = None, typ = "AND"):
    if type(name) != str:
        raise RuntimeError('AND gate name must be a string')
    if typ != "AND" and typ != "OR":
        raise RuntimeError('Gate type, typ, must be AND or OR')
        
    if typ == "AND":
        #source = 164
        source = 170
    elif typ == "OR":
        #source = 130
        source = 102
        
    if off_1 == None:
        off_1 = name + "_I1"
    if off_2 == None:
        off_2 = name + "_I2"
    if on_1 == None:
        on_1 = name + "_A1"
    if on_2 == None:
        on_2 = name + "_A2"
    
    
    
    S1_off = Species(name + "_INP1")
    S2_off = Species(name + "_INP2")
    S3_off = Species(name + "_OUT")
    So1_on = Species(name + "_SOU")

    S1 = Switch(S1_off, transcript = name + "_out_A", activator = on_1, inhibitor = off_1 )
    S2 = Switch(S2_off, transcript = name + "_out_A", activator = on_2, inhibitor = off_2 )
    S3 = Switch(S3_off, transcript = out, activator = name + "_out_A", inhibitor = name + "_out_I" )
    So1 = Source(So1_on, transcript = name + "_out_I")
    
    ic = {str(S1_off)+"_OFF": 500, "rna_"+on_1: 700, "rna_"+off_1: 200, str(S2_off)+"_OFF": 500, "rna_"+on_2: 700, "rna_"+off_2: 200, str(S3_off)+"_OFF": 500,
          "rna_"+name+"_out_A": 0, "rna_"+name+"_out_I": 0, str(So1_on):source, "protein_RNAP":100}
    
    return [S1,S2,S3,So1],ic
    
"""

In [ ]:
"""
comp,ic = GeneletGate("AND1","P")
M = Mixture(name = "Switch_test", components = comp, parameter_file = "default_parameters.txt")
repr(M)
CRN = M.compile_crn()
print(CRN)
"""

In [ ]:
"""
timepoints = np.linspace(0, 40, 1000)
R = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = ic)

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
p.circle(timepoints, R["AND1_INP1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_AND1_INP1_ON"], legend_label = "ON switch 1", color = "red")
p.circle(timepoints, R["complex_AND1_INP1_AI"],legend_label = "AI complex 1", color = "green")

s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["AND1_INP2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_AND1_INP2_ON"], legend_label = "ON switch 2" , color = "red")
s.circle(timepoints, R["complex_AND1_INP2_AI"],legend_label = "AI complex 2", color = "green")

q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["AND1_OUT_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_AND1_OUT_ON"], legend_label = "ON switch 3" , color = "red")

r = bokeh.plotting.figure(plot_width=300, plot_height=300)
r.circle(timepoints, R["rna_AND1_out_I"], legend_label = "Inhibitor 3", color = "blue")
r.circle(timepoints, R["complex_AND1_OUT_AI"], legend_label = "AI complex 3" , color = "green")
#p.circle(timepoints, R["complex_rna_A3_rna_I3"],legend_label = "AI complex 3", color = "green")
bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")
"""

In [25]:
S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")
S4_off = Species("Sw4")
#R3 = Species("I3",material_type="rna")
#R4 = Species("I4",material_type="rna")

S1 = Switch(S1_off, transcript = "I3", activator = "A1", inhibitor = "I1" )
S2 = Switch(S2_off, transcript = "I3", activator = "A2", inhibitor = "I2" )
S3 = Switch(S3_off, transcript = "I4", activator = "A3", inhibitor = "I3" )
#S4 = Switch(S4_off, transcript = "P", activator = "A4", inhibitor = "I4" )

M = Mixture(name = "Switch_test", components = [S1,S2,S3], parameter_file = "default_parameters.txt")

#rxn3 = Reaction([R3],[], k=12)
#rxn4 = Reaction([R4],[], k=12)
repr(M)
CRN = M.compile_crn()
#CRN.add_reactions([rxn3,rxn4])
print(CRN)

Species = Sw1_OFF, rna_I3, dna_A1, rna_I1, protein_RNAP, protein_RNAseH, Sw2_OFF, dna_A2, rna_I2, Sw3_OFF, rna_I4, dna_A3
Reactions = [
	Sw1_OFF + dna_A1 --> complex_Sw1_ON        massaction: k_f(Sw1_OFF,dna_A1)=0.0394*Sw1_OFF*dna_A1
	complex_Sw1_ON + rna_I1 --> Sw1_OFF + complex_Sw1_AI        massaction: k_f(complex_Sw1_ON,rna_I1)=0.696*complex_Sw1_ON*rna_I1
	dna_A1 + rna_I1 --> complex_Sw1_AI        massaction: k_f(dna_A1,rna_I1)=0.696*dna_A1*rna_I1
	complex_Sw1_ON + protein_RNAP <--> complex_Sw1_ON_protein_RNAP        massaction: k_f(complex_Sw1_ON,protein_RNAP)=10.0*complex_Sw1_ON*protein_RNAP k_r(complex_Sw1_ON_protein_RNAP)=0.1*complex_Sw1_ON_protein_RNAP
	complex_Sw1_ON_protein_RNAP --> complex_Sw1_ON + rna_I3 + protein_RNAP        massaction: k_f(complex_Sw1_ON_protein_RNAP)=0.064*complex_Sw1_ON_protein_RNAP
	Sw1_OFF + protein_RNAP <--> complex_Sw1_OFF_protein_RNAP        massaction: k_f(Sw1_OFF,protein_RNAP)=10.0*Sw1_OFF*protein_RNAP k_r(complex_Sw1_OFF_protein_RNAP)=0.1*compl

In [101]:
io = {"Sw1_OFF": 2500, "dna_A1": 5000, "rna_I1": 0, "Sw2_OFF": 2500, "dna_A2": 5000, "rna_I2": 0, "Sw3_OFF":700, "dna_A3": 1000, "rna_I3": 0, "protein_RNAseH":100, "protein_RNAP":500}
timepoints = np.linspace(0, 500, 1000)
R = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = io)
#plt.plot(timepoints, R["rna_A3"], label = "Product: A3", color = "red")
#plt.plot(timepoints, R["dna_S1"], label = "Switch OFF", color = "green")
#plt.plot(timepoints, R["complex_A1_dna_S1"], label = "Switch ON", color = "magenta")
#plt.plot(timepoints, R["complex_A1_I1"], label = "AI complex", color = "blue")
#plt.legend()

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
#p.circle(timepoints, R["rna_A3"], legend_label = "Activator 3")
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")
#p.circle(timepoints, R["complex_Sw1_AI"],legend_label = "AI complex 1", color = "green")
#p.legend.location = "center_right"
s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")
#s.circle(timepoints, R["complex_Sw2_AI"],legend_label = "AI complex 2", color = "green")
#s.legend.location = "center_right"

q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON"], legend_label = "ON switch 3" , color = "red")
#q.legend.location = "center_right"

#t = bokeh.plotting.figure(plot_width=300, plot_height=300)
#t.circle(timepoints, R["Sw4_OFF"], legend_label = "OFF switch 4", color = "orange")
#t.circle(timepoints, R["complex_Sw4_ON"], legend_label = "ON switch 4" , color = "red")


#r = bokeh.plotting.figure(plot_width=300, plot_height=300)
#r.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "blue")
#r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "AI complex 3" , color = "green")
#p.circle(timepoints, R["complex_rna_A3_rna_I3"],legend_label = "AI complex 3", color = "green")
bokeh.io.show(row(p, s, q))
warnings.filterwarnings("ignore")

odeint failed with mxstep=500...

Loading BokehJS ...

In [100]:
bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=400, plot_height=400)
#p.circle(timepoints, R["rna_A3"], legend_label = "Activator 3")
p.circle(timepoints, R["dna_A3"], legend_label = "Activator 3", color = "orange")
p.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "red")
p.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 3", color = "blue")

#p.circle(timepoints, R["complex_Sw1_AI"],legend_label = "AI complex 1", color = "green")
#p.legend.location = "center_right"
s = bokeh.plotting.figure(plot_width=400, plot_height=400)
s.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "red")
#s.circle(timepoints, R["complex_Sw4_AI"], legend_label = "Complex 4", color = "blue")
#s.legend.location = "center_right"



bokeh.io.show(row(p, s))
warnings.filterwarnings("ignore")

Loading BokehJS ...

In [94]:
R["complex_Sw1_ON_protein_RNAP"]

0        0.000000
1        3.504370
2        6.130122
3        8.727905
4       11.298013
          ...    
995    249.993215
996    249.993277
997    249.993339
998    249.993401
999    249.993461
Name: complex_Sw1_ON_protein_RNAP, Length: 1000, dtype: float64

In [ ]:
bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=400, plot_height=400)
#p.circle(timepoints, R["rna_A3"], legend_label = "Activator 3")
p.circle(timepoints, R["rna_A3"], legend_label = "Activator 3", color = "orange")
p.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "red")
p.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 3", color = "blue")

#p.circle(timepoints, R["complex_Sw1_AI"],legend_label = "AI complex 1", color = "green")
#p.legend.location = "center_right"
s = bokeh.plotting.figure(plot_width=400, plot_height=400)
s.circle(timepoints, R["rna_A4"], legend_label = "Activator 4", color = "orange")
s.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "red")
s.circle(timepoints, R["complex_Sw4_AI"], legend_label = "Complex 4", color = "blue")
#s.legend.location = "center_right"



bokeh.io.show(row(p, s))
warnings.filterwarnings("ignore")